In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import itertools
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
def read_dimacs_cnf(filename):
    clauses = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith("c"):
                continue
            if line.startswith("p cnf"):
                num_vars, num_clauses = map(int, line.strip().split()[2:])
            else:
                clause = list(map(int, line.strip().split()[:-1]))
                clauses.append(clause)
    return num_vars, clauses

def draw_hypergraph(num_vars, clauses, threshold=0):
    G = nx.Graph()

    for var in range(1, num_vars + 1):
        G.add_node(var)

    for i, clause in enumerate(clauses):
        G.add_nodes_from(map(abs, clause))
        for pair in itertools.combinations(map(abs, clause), 2):
            G.add_edge(*pair, clause=i+1)

    nodes_to_remove = [node for node, degree in dict(G.degree()).items() if degree <= threshold]
    G.remove_nodes_from(nodes_to_remove)

    node_colors = []
    for node in G.nodes():
        color = 0
        for i, clause in enumerate(clauses):
            if abs(node) in map(abs, clause):
                color = i + 1
                break
        node_colors.append(color)

    pos = nx.spring_layout(G)
    
    nx.draw(G, pos, with_labels=True, font_weight='bold', node_color=node_colors, cmap=plt.cm.Blues, node_size=800)

In [ ]:
def interactive_hypergraph(num_vars, clauses, threshold=0):
    G = nx.Graph()

    for var in range(1, num_vars + 1):
        G.add_node(var)

    for i, clause in enumerate(clauses):
        G.add_nodes_from(map(abs, clause))
        for pair in itertools.combinations(map(abs, clause), 2):
            G.add_edge(*pair, clause=i+1)

    nodes_to_remove = [node for node, degree in dict(G.degree()).items() if degree <= threshold]
    G.remove_nodes_from(nodes_to_remove)

    pos = nx.spring_layout(G)

    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += (x0, x1, None)
        edge_trace['y'] += (y0, y1, None)

    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers+text',
        hoverinfo='text',
        marker=dict(
            colorscale='Blues',
            reversescale=False,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Stopień węzła',
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=2)))

    for node in G.nodes():
        x, y = pos[node]
        node_trace['x'] += (x,)
        node_trace['y'] += (y,)
        node_trace['text'] += ('Var: ' + str(node),)

    for node, adjacencies in enumerate(G.adjacency()):
        node_trace['marker']['color'] += (len(adjacencies[1]),)

    layout = go.Layout(
        titlefont=dict(size=16),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=5, l=5, r=5, t=5),
        annotations=[dict(
            text="",
            showarrow=False,
            xref="paper", yref="paper")],
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

    fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
    fig.show()

In [ ]:
def update_graph(fig, G, pos, new_clauses=[], nodes_to_remove=[]):
    for clause in new_clauses:
        G.add_nodes_from(map(abs, clause))
        for pair in itertools.combinations(map(abs, clause), 2):
            G.add_edge(*pair, clause=len(G.edges())+1)

    G.remove_nodes_from(nodes_to_remove)

    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    fig.data[0].x = edge_x
    fig.data[0].y = edge_y

    node_x = []
    node_y = []
    node_text = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append('Var: ' + str(node))

    fig.data[1].x = node_x
    fig.data[1].y = node_y
    fig.data[1].text = node_text

    node_colors = [0] * len(G.nodes())
    for node, adjacencies in enumerate(G.adjacency()):
        node_colors[node] = len(adjacencies[1])

    fig.data[1].marker.color = node_colors

    return fig

In [ ]:
filename = "DIMACS_files/turbo_easy/example_2.cnf"
num_vars, clauses = read_dimacs_cnf(filename)

In [ ]:
draw_hypergraph(num_vars, clauses, threshold=8)

In [ ]:
interactive_hypergraph(num_vars, clauses, threshold=0)

In [ ]:
fig, G, pos = interactive_hypergraph(num_vars, clauses, threshold=0)
fig.show()

In [ ]:
# Update the graph dynamically (add/remove clauses or nodes)
# Example: add a new clause
new_clause = [1, 2, 3]
fig = update_graph(fig, G, pos, new_clauses=[new_clause])

# Example: remove nodes
nodes_to_remove = [1, 2]
fig = update_graph(fig, G, pos, nodes_to_remove=nodes_to_remove)

fig.show()